# I. Load Scores via ESPN API

In [2]:
from espn_functions import *
#Define week and year
week = 2
year = 2023
standings_df, matchup_df, HP_Owner, HP_Player, HP_Score, HT_Owner, HT_Score = run_espn_weekly(week, year)

Data not loaded correctly (Attempt 1 of 5), retrying in 1 second...
Loaded data from ESPN for week 2 of the 2023 season
Completed processing weekly scores and standings
Saved week 2 matchup data


# II. Create GPT Summary Email

In [8]:
summary = generate_summary(week, matchup_df, standings_df)
print(summary)

Hey there, fantasy football enthusiasts!

Week 2 of our league is officially in the books, and boy, oh boy, did we have some interesting matchups. Let's dive right in and dissect the madness, shall we?

First off, let's give a standing ovation to Kai Shen and his team, Not Sketch, who managed to eke out a win with a whopping 105.46 points. I'm not sure if there was some artistic inspiration behind his team name, but his performance was anything but sketchy. Well done, Kai!

And speaking of impressive performances, our league's resident "artist" John Qian and his team, fittingly named The Real Artist, secured their second consecutive victory with a commanding score of 118.8 points. Maybe he's got a secret paint-by-numbers strategy we don't know about.

But alas, not everyone can have a victorious week. I'm sorry to say, Richie Kay, but your team, Richie's ㄱㅅㄲ s, is currently begging to be placed in the nearest dumpster after scoring a total of 108.9 points. Richie, I'm not sure what tho

# III. Send G-Mail

In [12]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Load the user name and password from yaml file
user, password = config_data['gmail']['GMAIL_USER'], config_data['gmail']['GMAIL_PW']
recipients = list(config_data['emails']['espn_email_dict'].values())

# Initialize the server
s = smtplib.SMTP('smtp.gmail.com', 587)
s.starttls()
s.login(user, password)

#Format the tables
scoring_tabulate = tabulate(matchup_df, headers='keys', tablefmt='html', showindex=False)
standings_tabulate = tabulate(standings_df, headers='keys', tablefmt='html', showindex=False)

# Define message
subject = f"Weekly Fagudas FF Recap: Week {week}"
summary_html = summary.replace('\n', '<br>')
body = f"""\
<html>
  <head></head>
  <body>
    <br>{summary_html}</br>
    <br>
    <hr>
    <p><b>Highest Scoring Team</b>: {HT_Owner}: {HT_Score} points</p>
    <p><b>Highest Scoring Player</b>: {HP_Owner} - {HP_Player}: {HP_Score} Points</p>
    <br>
    {standings_tabulate}
  </body>
</html>
"""


#recipients = ['lithal06@yahoo.com']
sender = user

msg = MIMEMultipart("alternative")
msg["From"] = sender
msg["To"] = ", ".join(recipients)
msg["Subject"] = subject

# Attach the HTML body
msg.attach(MIMEText(body, 'html'))

# Send email
s.sendmail(sender, recipients, msg.as_string())

# Terminate the session
s.quit()

(221,
 b'2.0.0 closing connection i16-20020a05620a145000b00767721aebc0sm4424494qkl.32 - gsmtp')